# Create Slice for DYNAMOS in FABRIC 

This Jupyter notebook will create a FABRIC slice with resources that can be used for DYNAMOS in FABRIC.

### FABlib API References Examples

FABRIC API docs: https://fabric-fablib.readthedocs.io/en/latest/index.html

Example list of useful functions:
- [fablib.show_config](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.show_config)
- [fablib.list_sites](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_sites)
- [fablib.list_hosts](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.list_hosts)
- [fablib.new_slice](https://fabric-fablib.readthedocs.io/en/latest/fablib.html#fabrictestbed_extensions.fablib.fablib.FablibManager.new_slice)
- [slice.add_node](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.add_node)
- [slice.submit](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.submit)
- [slice.get_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.get_nodes)
- [slice.list_nodes](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.list_nodesß)
- [slice.show](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.show)
- [node.execute](https://fabric-fablib.readthedocs.io/en/latest/node.html#fabrictestbed_extensions.fablib.node.Node.execute)
- [slice.delete](https://fabric-fablib.readthedocs.io/en/latest/slice.html#fabrictestbed_extensions.fablib.slice.Slice.delete)


## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](./configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

After following all steps of the Configuring Environment notebook, you should be able to run this notebook without additional steps.

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

## Step 2: Setup the Environment for this Notebook

### Step 2.1: Import FABRIC API and other libraries

In [ ]:
import datetime

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();


### (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity and other settings or available resources.

In [ ]:
# Comment out lists you do not want to view

# List available sites
# fablib.list_sites()
# List available hosts
# fablib.list_hosts(includes=["CERN", "AMST"])
# List available images that can be used for the nodes
fablib.get_image_names()

### Step 2.2: Configure the parameters and variables
Can be used to set the corresponding slice and other variables used for subsequent cells.

In [ ]:
# Name of the slice
slice_name = 'DYNAMOS'
# Set image to ubuntu specific (used in FABRIC Kubernetes cluster example)
image = "default_ubuntu_20"
# Site used for the nodes (Use the UvA site: AMST)
site = "AMST"
# Host of the site used for the nodes (Use specific host to avoid error occurred without it: "Timeout waiting for the server to come up")
site_host = "amst-w1.fabric-testbed.net"
# Nodes:
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

## Step 3: Create the Slice and Nodes

The following creates a slice with the required nodes. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated. 

By default, the submit function will block until the node is ready and will display the progress of your slice being built (it will automatically retry until the slice is submitted (i.e. the state of the slice is not configuring anymore)).

Note: the slice must incorporate resources to be created, it cannot be created with no resources here, giving the error: "Exception: Submit request error: return_status Status.INVALID_ARGUMENTS, slice_reservations: Either topology None or slice graph None must be specified"



In [ ]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

# Add Nodes with the specific variables
# Also validate the node can be created and raise an exception in case of failure
node1 = slice.add_node(name=node1_name, site=site, validate=True, raise_exception=True, host=site_host, image=image)
node2 = slice.add_node(name=node2_name, site=site, validate=True, raise_exception=True, host=site_host, image=image)
node3 = slice.add_node(name=node3_name, site=site, validate=True, raise_exception=True, host=site_host, image=image)
# TODO: for now this is good I think: 1 control node and 2 worker nodes

# Calculate the lease end time for 2 weeks from now with timezone information
lease_end_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(weeks=2)

# Submit the slice, using an end date 2 weeks from now (the current maximum lease time) 
# to make sure that the slice can be used for a longer period of time. Progress shows an indicator of the current progression.
# Wait until the state is finished and use an interval (it may take some time before the slice and nodes are created)
slice.submit(wait=True, wait_timeout=3600, wait_interval=20, progress=True, wait_jupyter='text', lease_end_time=lease_end_time);

## Step 3: Observe the Slice's Attributes
This step just observes the slice's attributes. 

### Show the slice attributes 

In [ ]:
slice.show();

#### List the nodes

In [ ]:
slice.list_nodes();

### Run a test Experiment to Validate the nodes

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [ ]:
#node = slice.get_node('Node1')

for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`')